# Assignment Three
## Task C

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models 
from torchvision.utils import make_grid

import matplotlib.pyplot as plt

In [2]:
transform = transforms.ToTensor()

train_data = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST('./data', train=False, download=True, transform=transform)

In [3]:
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)
test_loader = DataLoader(test_data, batch_size=10, shuffle=False)

In [4]:
class CNNmodel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 3, 1)
        self.conv2 = nn.Conv2d(6, 16, 3, 1)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
            
        return torch.log_softmax(x, dim=1)


In [5]:
model = CNNmodel()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [6]:
import time
start = time.time()

epochs = 5

train_correct = []

for i in range(epochs):
    
    trn_corr = 0
    
    for b, (x_train, y_train) in enumerate(train_loader):
        
        b+=1
        
        y_pred = model(x_train)
        loss = criterion(y_pred, y_train)
        predicted = torch.max(y_pred.data, 1)[1]
        batch_corr = (predicted == y_train).sum()
        trn_corr += batch_corr
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if b%600==0:
            print(f'Epoch: {i} Loss: {loss} Accuracy: {trn_corr.item()*100/(b*10):7.3f}')
            
    train_correct.append(trn_corr)
        
full_time = time.time() - start
print(f'Training took {(full_time/60):2f} minutes')
    

Epoch: 0 Loss: 0.11220882087945938 Accuracy:  76.833
Epoch: 0 Loss: 0.06343716382980347 Accuracy:  85.358
Epoch: 0 Loss: 0.02166539989411831 Accuracy:  88.872
Epoch: 0 Loss: 0.054398320615291595 Accuracy:  90.692
Epoch: 0 Loss: 0.6612063646316528 Accuracy:  91.833
Epoch: 0 Loss: 0.011352772824466228 Accuracy:  92.686
Epoch: 0 Loss: 0.005288996268063784 Accuracy:  93.367
Epoch: 0 Loss: 0.21493253111839294 Accuracy:  93.852
Epoch: 0 Loss: 0.42051681876182556 Accuracy:  94.228
Epoch: 0 Loss: 0.008206434547901154 Accuracy:  94.527
Epoch: 1 Loss: 0.00038428528932854533 Accuracy:  97.600
Epoch: 1 Loss: 0.003921263851225376 Accuracy:  97.708
Epoch: 1 Loss: 0.00021461513824760914 Accuracy:  97.772
Epoch: 1 Loss: 0.0023190572392195463 Accuracy:  97.888
Epoch: 1 Loss: 0.0032765227369964123 Accuracy:  97.910
Epoch: 1 Loss: 0.009429538622498512 Accuracy:  97.933
Epoch: 1 Loss: 0.001287441817112267 Accuracy:  97.960
Epoch: 1 Loss: 0.0013194407802075148 Accuracy:  98.019
Epoch: 1 Loss: 0.02776058577